In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import SGD
import tensorflow
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
import sklearn
from sklearn.metrics import confusion_matrix,classification_report

import time

start_time = time.time()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

KeyboardInterrupt: 

**Changing the directory to data2 folder**

In [ ]:

os.chdir("standard-ocr-dataset/data")

### Importing Necessary Modules 

### Creating a Function to plot images 

In [ ]:
### defining some function to make our work easier


# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plot_images(images_arr, imageWidth, imageHeight):
    fig, axes = plt.subplots(1, 5, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img.reshape(imageWidth, imageHeight), cmap="gray")
        ax.axis("off")
    plt.tight_layout()
    plt.show()

##### Some constants to be defined 

In [ ]:
batch_size = 32
epochs = 50
IMG_HEIGHT = 28
IMG_WIDTH = 28

### Image Data Generator 
We defined two data generators, one that augments the data to make our training more general and one that just scales and centers the data.

In [ ]:
def preprocessing_fun(img):
    #     print(img.shape)
#     print(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY, CV_8UC1)
    img = cv2.GaussianBlur(img,(3,3),0)
    img = img.reshape((28,28,1))
    thresh = cv2.adaptiveThreshold(img, 255, 1, 1, 11, 2)
    print(thresh.shape)
    #img = img.reshape((28,28,1))
    img = np.where(img>140,1,0)
    img  = img/255
    #return img

In [ ]:
augmented_image_gen = ImageDataGenerator(
        rescale = 1/255.0,
    rotation_range=2,
    width_shift_range=.1,
    height_shift_range=.1,
    zoom_range=0.1,
    shear_range=2,
    brightness_range=[0.9, 1.1],
    validation_split=0.2,
   
   )

normal_image_gen = ImageDataGenerator(
    rescale = 1/255.0,
    validation_split=0.2,
  
   )


#### Using Data Generator generate batches to train our model

In [ ]:
train_data_gen = augmented_image_gen.flow_from_directory(batch_size=batch_size,
                                                     directory="./training_data",
                                                     color_mode="grayscale",
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode="categorical",
                                                     seed=65657867,
                                                     subset='training')
val_data_gen = normal_image_gen.flow_from_directory(batch_size=batch_size,
                                                     directory="./testing_data",
                                                     color_mode="grayscale",
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode="categorical",
                                                     seed=65657867,
                                                     subset='validation')


#### samples

In [3]:

sample_training_images, _ = next(train_data_gen)
plot_images(sample_training_images[:7], IMG_WIDTH, IMG_HEIGHT)

NameError: name 'train_data_gen' is not defined

### Defining our Sequential model 

In [ ]:
# model = Sequential([
#     Conv2D(16, 3, 
#            padding='same',
#            activation='relu',
#            kernel_regularizer=regularizers.l2(0.0001),
#            input_shape=(IMG_HEIGHT, IMG_WIDTH , 1)),
#     MaxPooling2D(),
#     Dropout(0.2),
#     Flatten(),
#     Dense(
#         50,
#         activation='relu',
#         kernel_regularizer=regularizers.l2(0.0001)
#     ),
#     Dropout(0.2),
#     Dense(36, activation='softmax')
# ])

# define cnn model
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(36, activation='softmax'))
    return model
#     # compile model
# 	opt = SGD(lr=0.01, momentum=0.9)
# 	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
# 	return model

In [ ]:
model = define_model()

In [ ]:
import tensorflow
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
#Prepare call backs
EarlyStop_callback = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

checkpoint_dir = 'path/to/checkpoints'  # Specify the checkpoint directory
checkpoint_path = os.path.join(checkpoint_dir, 'model_checkpoint.h5')

# Create the directory if it does not exist
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Define the callback for model checkpointing
checkpoint = ModelCheckpoint(
    checkpoint_path, monitor='val_loss', mode='min', save_best_only=True
)
lr = ReduceLROnPlateau(monitor = 'val_loss',factor = 0.5,patience = 3,min_lr = 0.00001)
my_callbacks = [EarlyStop_callback, checkpoint, lr]

### Actual training of our model

In [ ]:

model.compile(optimizer=SGD(lr=0.01, momentum=0.9),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['categorical_accuracy'])

history = model.fit(
    train_data_gen,
    steps_per_epoch=train_data_gen.samples // batch_size,
    epochs=32,
    validation_data=val_data_gen,
    validation_steps=val_data_gen.samples // batch_size,
    callbacks=my_callbacks
)

In [ ]:
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

loss = history.history['loss']



val_loss = history.history['val_loss']

epochs_range = range(32)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Test our Model

In [ ]:
normal_image_gen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True
)
test_data_gen = normal_image_gen.flow_from_directory(batch_size=5193,
                                                     directory="./testing_data",
                                                     color_mode="grayscale",
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode="categorical")

In [ ]:
from PIL import Image
test_images, test_labels = next(test_data_gen)
filenames = test_data_gen.filenames
test_pred = model.predict(test_images)

In [ ]:

true_labels = tf.argmax(test_labels,1)

In [ ]:
predicted_labels = tf.argmax(test_pred,1)

In [ ]:
confusion_matrix(true_labels, predicted_labels)

In [4]:
img = cv2.imread('testing_data/N/28333.png',0)
img = img/255
img = cv2.resize(img, (28,28))
img = img.reshape((1,28,28,1))
tf.argmax(model.predict(img),1)

[ WARN:0@18912.071] global loadsave.cpp:248 findDecoder imread_('testing_data/N/28333.png'): can't open/read file: check file path/integrity


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

### Perfect ! 

this model works well but somehow 0, O is bit confusing for this model as we know

In [ ]:
model.save("ocr.h5")
elapsed_time = time.time() - start_time

In [ ]:
print(elapsed_time)